In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize
pd.set_option('display.max_colwidth', None)

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

d:\Anaconda\envs\NLP\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_new_df(df):
    new_df = pd.DataFrame(columns=['Answer', 'Index'])
    texts = []
    indexes = []
    for index in range(len(df)):
        text_list = sent_tokenize(df['Answer'][index])
        for text in text_list:
            texts.append(text)
            indexes.append(index)

    new_df['Answer'] = pd.Series(texts)
    new_df['Index'] = pd.Series(indexes)
    return new_df

In [3]:
df = pd.read_csv("./data/FAQs.csv")
test_df = pd.read_csv("./data/FAQs_test.csv")
new_df = get_new_df(df)

In [4]:
words = new_df['Answer'][4]